In [135]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from inltk.inltk import setup
from inltk.inltk import tokenize
from collections import Counter
import pandas as pd
import re
from langdetect import detect
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from collections import Counter
import seaborn as sns
import os


class HindiRecommender:
    def __init__(self):
        sns.set(rc={'figure.figsize':(10,7)})
        pd.set_option('max_colwidth', 400)
        self.df = pd.read_csv(os.path.join(os.getcwd(), 'data/clean_data.csv'))
        self.df.drop(['Unnamed: 0'] , axis = 1, inplace = True)
        print(self.df.columns)
        self.df = self.df.drop_duplicates(subset='artist', keep='first')
        self.df = self.df.reset_index(level=0)
        self.df.rename(columns = {'Song name'  : 'song_name'}, inplace = True)

        self.train()

    # Calculating cosine similarities from lyrics and storing similar song results in results dict

    def train(self):
        self.tf = TfidfVectorizer(analyzer='word', min_df=0, max_features= 100 , lowercase=True)
        self.tfidf_matrix = self.tf.fit_transform(self.df['lyrics'])

        self.cosine_similarities = linear_kernel(self.tfidf_matrix, self.tfidf_matrix)
        self.results = {}

        for idx, row in self.df.iterrows():
            similar_indices = self.cosine_similarities[idx].argsort()[:-100:-1]
            similar_items = [(self.cosine_similarities[idx][i], self.df['id'][i]) for i in similar_indices]
            self.results[row['id']] = similar_items[1:]



    def item(self,id):
        return self.df.loc[self.df['id'] == id]['song_name']

    def recommend(self, artist, song_name, num):
        # filter df to find id of the song with name and artist
        song = self.df.query('song_name == @song_name and artist == @artist')
        # if doesnt exist, return False for now TODO : change this..
        # if exists, return the recommendations
        if(not song.empty):
            id = song['id']
            print(id.item())
            id = id.item()
            print("Recommending " + str(num) + " songs similar to " + self.item(id))
            recs = self.results[id][:num]
            i=0
            for rec in recs:
                print("We recommend : " + self.item(rec[1]) + " (score:" + str(rec[0]) + ")")
        else:
            # get song data from api
            # if data not found, return error
            # make sure lyrics is in hindi script
            
            # add to the data 
            #self.addRow( songData )
            
            # call the same function with same arguments and return
            #return(self.recommend(artist, song_name, num))
            print("Song not found in the dataset")

    
    def addRow(self, song_name, type, artist, lyrics):
        id = len(self.df['id'])
        newRow = [0,song_name, type, artist, lyrics, id]
        print(len(self.df.loc[len(self.df['id']) - 1]))
        # add the new song data to the df
        self.df.loc[len(self.df['id'])] = newRow
        #train again
        self.train()
        

In [134]:
t.recommend('arijit singh', 'kesariya', 3)

159
159    Recommending 3 songs similar to kesariya
Name: song_name, dtype: object
30    We recommend : ishk bina (score:0.6170150210459446)
Name: song_name, dtype: object
29    We recommend : isk da mara (score:0.5914068376696214)
Name: song_name, dtype: object
80    We recommend : tara gin gin gin gin gin gin gin gin gin gin gin gin gin gin gin gin gin gin gin gin gin gin gin gin gin gin gin gin gin gin gin gin gin gin gin gin gin gin gin gin gin gin gin gin gin) (score:0.5537321882573178)
Name: song_name, dtype: object


In [131]:
t = HindiRecommender()

Index(['song_name', 'type', 'artist', 'lyrics', 'id'], dtype='object')


In [132]:
t.addRow(song_name = 'kesariya', type = 'romantic', artist =  'arijit singh', lyrics =  '''मुझको... इतना बताये कोई कैसे तुझसे दिल ना लगाए कोई रब्बा ने तुझको बनाने में कर दी है हुस्न की खाली तिजोरियाँ काजल की सिहाई से लिखी है तूने जाने कितनो की लव स्टोरियाँ केसरिया तेरा इश्क़ है पिया रंग जाऊं जो मैं हाथ लगाऊँ दिन बीते सारा तेरी फ़िक्र में रैन सारी तेरी खैर मनाऊँ शिवा... ईशा का मतलब जानते हो तुम ? पार्वती अब शिवा का साथ पार्वती नही देगी तो कौन देगा पतझड़ के मौसम में भी रंगी चनारो जैसी झनके सन्नाटो में तू वीना के तारो जैसी... हम्म... सदियों से भी लम्बी ये मन की अमावसे है और तू फुलझड़ियों वाले त्योहारों जैसी चंदा भी दीवाना है तेरा जलती है तुझसे सारी चकोरियाँ... काजल की सिहाई से लिखी है तूने जाने कितनो की लव स्टोरियाँ... लव स्टोरियाँ... केसरिया तेरा इश्क़ है पिया रंग जाऊं जो मैं हाथ लगाऊँ दिन बीते सारा तेरी फ़िक्र में रैन सारी तेरी खैर मनाऊँ केसरिया तेरा इश्क़ है पिया इश्क़ है पिया केसरिया तेरा इश्क़ है पिया इश्क़ है पिया केसरिया तेरा''')

6


In [111]:
#t.addRow([0,'new','new', 'new', 'new', 'new'])
df = pd.read_csv('data/transliterated_data.csv')
df.drop(['level_0', 'Unnamed: 0'], axis = 1, inplace = True)

In [112]:
df.columns

Index(['song_name', 'type', 'artist', 'lyrics', 'id'], dtype='object')

In [113]:
df.to_csv('data/clean_data.csv')

In [64]:
artist = "mehtab"
song_name = "har jani a"
t.recommend(artist, song_name,3)

781
156    Recommending 3 songs similar to har jani a
Name: song_name, dtype: object
151    We recommend : hammen tummen (score:0.5973167093264719)
Name: song_name, dtype: object
71    We recommend : jabra phan (score:0.5861383384971809)
Name: song_name, dtype: object
97    We recommend : dil todeya (score:0.531430640036588)
Name: song_name, dtype: object


In [87]:

df.query('song_name == @song_name and artist == @artist')

,level_0,song_name,type,artist,lyrics,id
1,1,akkad bakkar,party,badshah,"अक्कड़ बक्कड़ बॉम्बे बो 80, 90 पुरे 100 रात के बज गए पौने दो जो होता है होने दो अक्कड़ बक्कड़ बॉम्बे बो 80, 90 पुरे 100 रात के बज गए पौने दो जो होता है होने दो अक्कड़ बक्कड़ बॉम्बे बो 80, 90 पुरे 100 रात के बज गए पौने दो जो होता है होने दो अक्कड़ बक्कड़ बॉम्बे बो 80, 90 पुरे 100 रात के बज गए पौने दो जो होता है होने दो अक्कड़ बक्कड़ अक्कड़ बक्कड़ अक्कड़ बक्कड़ अक्कड़ बक्कड़.. कुड़ी तू इक लोनल...",2


In [14]:
from indicate import transliterate
english_translated = transliterate.hindi2english("हिंदी")
print(english_translated)

hindi


In [ ]:
# train with hindi
# search with transliterated
# use id as common attribute

In [18]:
# transliterate song_name and artist to english for easier search
newDf = []
for i in df['song_name']:
    newDf.append(transliterate.hindi2english(i))
    
newDf = []
for i in df['artist']:
    newDf.append(transliterate.hindi2english(i))
df['artist'] = newDf 

Not able to transliterate अपना टाइम आएगा within 10 seconds, exiting with Function transliterate (args=('अपना टाइम आएगा', 1024, <keras.preprocessing.text.Tokenizer object at 0x0000019ADEBAEA00>, <keras.preprocessing.text.Tokenizer object at 0x0000019ADEBAE5E0>, <indicate.encoder.Encoder object at 0x0000019ADEBAE0A0>, <indicate.decoder.Decoder object at 0x0000019AE5D29370>, 47)) (kwargs={}) timed out after 10.000000 seconds.



ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "c:\users\aleena\appdata\local\programs\python\python38\lib\site-packages\tensorflow_addons\seq2seq\decoder.py", line 508, in body
    return (  File "c:\users\aleena\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\util\nest.py", line 916, in map_structure
    return pack_sequence_as(  File "c:\users\aleena\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\util\nest.py", line 917, in <listcomp>
    structure[0], [func(*x) for x in entries],  File "c:\users\aleena\appdata\local\programs\python\python38\lib\site-packages\tensorflow_addons\seq2seq\decoder.py", line 506, in <lambda>
    lambda ta, out: ta.write(time, out), outputs_ta, emit  File "c:\users\aleena\appdata\local\programs\python\python3

Not able to transliterate चलो चले रे साजन धीरे धीरे within 10 seconds, exiting with Function transliterate (args=('चलो चले रे साजन धीरे धीरे', 1024, <keras.preprocessing.text.Tokenizer object at 0x0000019ADEBAEA00>, <keras.preprocessing.text.Tokenizer object at 0x0000019ADEBAE5E0>, <indicate.encoder.Encoder object at 0x0000019ADEBAE0A0>, <indicate.decoder.Decoder object at 0x0000019AE5D29370>, 47)) (kwargs={}) timed out after 10.000000 seconds.



ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "c:\users\aleena\appdata\local\programs\python\python38\lib\site-packages\tensorflow_addons\seq2seq\decoder.py", line 508, in body
    return (  File "c:\users\aleena\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\util\nest.py", line 916, in map_structure
    return pack_sequence_as(  File "c:\users\aleena\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\util\nest.py", line 917, in <listcomp>
    structure[0], [func(*x) for x in entries],  File "c:\users\aleena\appdata\local\programs\python\python38\lib\site-packages\tensorflow_addons\seq2seq\decoder.py", line 506, in <lambda>
    lambda ta, out: ta.write(time, out), outputs_ta, emit  File "c:\users\aleena\appdata\local\programs\python\python3

Not able to transliterate जग सुना सुना जग सुना सुना  within 10 seconds, exiting with Function transliterate (args=('जग सुना सुना जग सुना सुना ', 1024, <keras.preprocessing.text.Tokenizer object at 0x0000019ADEBAEA00>, <keras.preprocessing.text.Tokenizer object at 0x0000019ADEBAE5E0>, <indicate.encoder.Encoder object at 0x0000019ADEBAE0A0>, <indicate.decoder.Decoder object at 0x0000019AE5D29370>, 47)) (kwargs={}) timed out after 10.000000 seconds.



...

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "c:\users\aleena\appdata\local\programs\python\python38\lib\site-packages\tensorflow_addons\seq2seq\decoder.py", line 508, in body
    return (  File "c:\users\aleena\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\util\nest.py", line 916, in map_structure
    return pack_sequence_as(  File "c:\users\aleena\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\util\nest.py", line 917, in <listcomp>
    structure[0], [func(*x) for x in entries],  File "c:\users\aleena\appdata\local\programs\python\python38\lib\site-packages\tensorflow_addons\seq2seq\decoder.py", line 506, in <lambda>
    lambda ta, out: ta.write(time, out), outputs_ta, emit  File "c:\users\aleena\appdata\local\programs\python\python3

Not able to transliterate तुम्हारी पलकों की चिलमनो में within 10 seconds, exiting with Function transliterate (args=('तुम्हारी पलकों की चिलमनो में', 1024, <keras.preprocessing.text.Tokenizer object at 0x0000019ADEBAEA00>, <keras.preprocessing.text.Tokenizer object at 0x0000019ADEBAE5E0>, <indicate.encoder.Encoder object at 0x0000019ADEBAE0A0>, <indicate.decoder.Decoder object at 0x0000019AE5D29370>, 47)) (kwargs={}) timed out after 10.000000 seconds.



ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "c:\users\aleena\appdata\local\programs\python\python38\lib\site-packages\tensorflow_addons\seq2seq\decoder.py", line 508, in body
    return (  File "c:\users\aleena\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\util\nest.py", line 916, in map_structure
    return pack_sequence_as(  File "c:\users\aleena\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\util\nest.py", line 917, in <listcomp>
    structure[0], [func(*x) for x in entries],  File "c:\users\aleena\appdata\local\programs\python\python38\lib\site-packages\tensorflow_addons\seq2seq\decoder.py", line 506, in <lambda>
    lambda ta, out: ta.write(time, out), outputs_ta, emit  File "c:\users\aleena\appdata\local\programs\python\python3

Not able to transliterate दिल मेरा चुराया क्यूँ within 10 seconds, exiting with Function transliterate (args=('दिल मेरा चुराया क्यूँ', 1024, <keras.preprocessing.text.Tokenizer object at 0x0000019ADEBAEA00>, <keras.preprocessing.text.Tokenizer object at 0x0000019ADEBAE5E0>, <indicate.encoder.Encoder object at 0x0000019ADEBAE0A0>, <indicate.decoder.Decoder object at 0x0000019AE5D29370>, 47)) (kwargs={}) timed out after 10.000000 seconds.



ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "c:\users\aleena\appdata\local\programs\python\python38\lib\site-packages\tensorflow_addons\seq2seq\decoder.py", line 508, in body
    return (  File "c:\users\aleena\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\util\nest.py", line 916, in map_structure
    return pack_sequence_as(  File "c:\users\aleena\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\util\nest.py", line 917, in <listcomp>
    structure[0], [func(*x) for x in entries],  File "c:\users\aleena\appdata\local\programs\python\python38\lib\site-packages\tensorflow_addons\seq2seq\decoder.py", line 506, in <lambda>
    lambda ta, out: ta.write(time, out), outputs_ta, emit  File "c:\users\aleena\appdata\local\programs\python\python3

In [43]:
filtered = df[df['song_name'] != '' ]
fullyFiltered = filtered[filtered['artist'] != '']
fullyFiltered.to_csv('transliterated_data.csv')

In [50]:
a = 'hello'


In [56]:
df

,level_0,song_name,type,artist,lyrics,id
0,0,neelanillaiyambar,romantic,kishor kumar,नीले नीले अम्बर पर चाँद जब आये प्यार बरसाए हमको तरसाए नीले नीले अम्बर पर चाँद जब आये प्यार बरसाए हमको तरसाए ऐसा कोई साथी हो ऐसा कोई प्रेमी हो प्यास दिल की बुझा जाए ऊँचे ऊँचे पर्बत जब चुमते हैं अम्बर को प्यासा-प्यासा अम्बर जब चूमता है सागर को प्यार से कसने को बाहों में बसने को दिल मेरा ललचाये कोई तो आ जाए ऐसा कोई साथी हो ऐसा कोई प्रेमी हो प्यास दिल की बुझा जाए ठन्डे-ठन्डे झोंके जब बालों क...,1
1,1,akkad bakkar,party,badshah,"अक्कड़ बक्कड़ बॉम्बे बो 80, 90 पुरे 100 रात के बज गए पौने दो जो होता है होने दो अक्कड़ बक्कड़ बॉम्बे बो 80, 90 पुरे 100 रात के बज गए पौने दो जो होता है होने दो अक्कड़ बक्कड़ बॉम्बे बो 80, 90 पुरे 100 रात के बज गए पौने दो जो होता है होने दो अक्कड़ बक्कड़ बॉम्बे बो 80, 90 पुरे 100 रात के बज गए पौने दो जो होता है होने दो अक्कड़ बक्कड़ अक्कड़ बक्कड़ अक्कड़ बक्कड़ अक्कड़ बक्कड़.. कुड़ी तू इक लोनल...",2
2,2,akhiya,sad,popon,ओ थक गेया अँखियाँ ओ जग्ग दियाँ अखियाँ माहियाँ ना आया लभदियाँ अँखियाँहाये प्यार दियाँ क़समाँ खा के छड़ गया तू इश्क़ भुला के दस्स जा तू मेरा की क़ुसूर वे तू अखियाँ नू कहने दे तू अखियाँ नू बहने दे जो दर्द सतावें तेनू ना दिल विच रहने दे तू ग़ल्ल कन्नु समझावे प्यार किन्ना तड़पावे जब तक ना आवे माही तू अखियाँ नू रोने दे ल ला ला ला.. तेरे वे तेरे इंतेज़ार में दिल हो गया फ़क़ीरा प्यार में तुझको भुला...,3
3,3,angase anga lagana,romantic,alaka yaganik,"अरे जो जी में आए.. अरे जो जी में आए.. तुम आज कर लो चाहो जिसे इन बाँहो में भर लो अंग से अंग लगाना सजन हमें ऐसे रंग लगाना अंग से अंग लगाना सजन हमें ऐसे रंग लगाना गालों से ये गाल लगा के नैनों से ये नैन मिला के होली आज मनाना सजन हमें ऐसे रंग लगाना अंग से अंग लगाना सजन हमें ऐसे रंग लगाना अंग से अंग लगाना सजन हमें ऐसे रंग लगाना ऊपर ऊपर रंग लगय्यो, ना करिओ कुछ नीचे मोसे कुछ ना बोल, खड़ी रे चुप स...",4
4,4,agar zindagi has,romantic,asha bhosale,अगर ज़िन्दगी हो तो तेरे संग हो अगर ज़िन्दगी हो तो तेरे संग हो अगर मौत हो तो वो हो तुझसे पहले अगर मौत हो तो वो हो तुझसे पहले अगर ज़िन्दगी हो तो तेरे संग हो अगर ज़िन्दगी हो तो तेरे संग हो अगर मौत हो तो वो हो तुझसे पहले अगर मौत हो तो वो हो तुझसे पहले बलमा.. बलमा.. बलमा.. बलमा.. अब दिल लगता कही ना तेरे बिना अब दिल लगता कही ना तेरे बिना मुश्किल होगा जीना तेरे बिना आ मिलके वादा करले जब तक होगा दम मे...,5
...,...,...,...,...,...,...
255,780,har jani a,sad,mehtab,कई वारि सोचा ऐवे कादा प्यार हो गया होया वि ता होया इन्ना ज्यादा प्यार हो गया ओहदे बिना रहना लग्गे बड़ा औखा चुप रह के सार जानी आ झट आँख देंदे तोड़ देनी यारी ओथे फे मैं हार जानी आ झट आँख देंदे तोड़ देनी यारी ओथे फे मैं हार जानी आ इक दो वारि गल बंद जदो होई सी ओहदे कॉड जाके हाथ जोड़ जोड़ रोई सी मैं ओहदे कॉड जाके हाथ जोड़ जोड़ रोई सी ओहनू लाग गयीअ मेरी कमजोरी ताही मैं सहार जानी आ झट आँख देंदे तोड़ देनी य...,781
256,784,hukka bar,party,,हो ओ … तेरी अखियों का वार जैसे शेर का शिकार तेरा हुस्न धुयंदार जैसे जलता सिगार तेरी अखियों का वार जैसे शेर का शिकार तेरा हुस्न धुयंदार जैसे जलता सिगार तेरे प्यार का नशा कभी आर कभी पार तेरा प्यार प्यार प्यार हुक्का बार तेरा प्यार प्यार प्यार हुक्का बार तेरा प्यार प्यार प्यार हुक्का बार बार बार तेरा प्यार प्यार प्यार हुक्का बार तेरा प्यार प्यार प्यार हुक्का बार तेरा प्यार प्यार प्यार हुक्का बार ...,785
257,785,husna parcham,party,raja kumari,"हाथ में रात ये बाकी है बात नापाक वफ़ा की है संदली सी पुरवाई साथ है ओ तू है नाकाम मोहब्बत में मैं जलूं प्यार की सिद्दत में धुंदली सी रुसवाई साथ है काम तू आजा मेरे सोच ना यार सवेरे छोड़ दूंगी मैं घर पे करता है क्यूँ गम हुस्न परचम, हुस्न परचम लहराया यार दमा-दम-दम हुस्न परचम, हुस्न परचम लहराया यार दमा-दम-दम हुस्न परचम, हुस्न परचम लहराया यार दिखा दम-ख़म हुस्न परचम.. Bang like a drum work my body आज रा...",786
258,787,ha gujarish,romantic,sonou neegam,हूँमम.. ओ.. तू मेरी अधूरी प्यास प्यास तू आ गयी मन को रास रास अब तो.. तू मेरी अधूरी प्यास प्यास तू आ गयी मन को रास रास अब तो तू आजा पास पास है गुजारिश है हाल तो दिल का तंग तंग तू रंग जा मेरे रंग रंग बस चलना मेरे संग संग है गुजारिश कह दे तू हाँ तो जिंदगी झरनों सी छूट के हँसेगी मोती होंगे मोती राहों में येई येई येई तू मेरी अधूरी प्यास प्यास तू आ गयी मन को रास रास अब तो तू आजा पास पास है गुजार...,788
